In [20]:
games_norm_df = sqlContext.read.parquet("./games_norm_df_p/")
games_norm_df.registerTempTable("games")

distinct_games_weather_df = sqlContext.read.parquet("./weather_p/")
distinct_games_weather_df.registerTempTable("weather")

connection_d = {'url':'jdbc:postgresql://bluemix-sandbox-dal-9-portal.2.dblayer.com:15656/compose',
                'dbtable':'play_player',
                'user':'admin',
                'password':'RFSUPYGYXCMNKGUN'}  

connection_d["dbtable"] = "drive"
drive_df = sqlContext.read.format('jdbc').options(**connection_d).load()
drive_df.registerTempTable("drive")

connection_d["dbtable"] = "game"
play_df = sqlContext.read.format('jdbc').options(**connection_d).load()
play_df.registerTempTable("game")


connection_d["dbtable"] = "play"
play_df = sqlContext.read.format('jdbc').options(**connection_d).load()
play_df.registerTempTable("play")


connection_d["dbtable"] = "agg_play"
play_df = sqlContext.read.format('jdbc').options(**connection_d).load()
play_df.registerTempTable("agg_play")

connection_d["dbtable"] = "team"
team_df = sqlContext.read.format('jdbc').options(**connection_d).load()
team_df.registerTempTable("team")

connection_d["dbtable"] = "player"
player_df = sqlContext.read.format('jdbc').options(**connection_d).load()
player_df.registerTempTable("player")

connection_d["dbtable"] = "play_player"
play_player_df = sqlContext.read.format('jdbc').options(**connection_d).load()
play_player_df.registerTempTable("play_player")

In [3]:
team_1 = 'ATL'
team_2 = 'NE'

In [4]:
!pwd

/gpfs/fs01/user/s2fc-631a29225e9565-5626aa220b46/notebook/work


In [5]:
query = """
select 
*
from games
where
TEAM = '{}'
or
TEAM = '{}'
limit 10
"""
query = query.format(team_1, team_2)
sqlContext.sql(query).toPandas()


,GSIS_ID,GAMEKEY,START_TIME,WEEK,DAY_OF_WEEK,SEASON_YEAR,SEASON_TYPE,FINISHED,LOCATION,HOME_GAME,...,OPPONENT_TEAM,OPPONENT_SCORE,OPPONENT_SCORE_Q1,OPPONENT_SCORE_Q2,OPPONENT_SCORE_Q3,OPPONENT_SCORE_Q4,OPPONENT_SCORE_Q5,OPPONENT_TURNOVERS,WON,TIE
0,2015110105,56610,2015-11-01 12:00:00,8,Sunday,2015,Regular,True,ATL,1,...,TB,23,3,10,7,0,3,2,0,0
1,2015081353,56762,2015-08-13 18:30:00,1,Thursday,2015,Preseason,True,NE,1,...,GB,22,3,6,6,7,0,2,0,0
2,2015112901,56666,2015-11-29 12:00:00,12,Sunday,2015,Regular,True,ATL,1,...,MIN,20,7,0,3,10,0,1,0,0
3,2015120612,56692,2015-12-06 15:25:00,13,Sunday,2015,Regular,True,NE,1,...,PHI,35,0,14,14,7,0,2,0,0
4,2015112202,56650,2015-11-22 12:00:00,11,Sunday,2015,Regular,True,ATL,1,...,IND,24,0,7,7,10,0,2,0,0
5,2009090352,54774,2009-09-03 18:30:00,4,Thursday,2009,Preseason,True,ATL,1,...,BAL,20,7,0,0,13,0,3,0,0
6,2009090354,54776,2009-09-03 18:30:00,4,Thursday,2009,Preseason,True,NE,1,...,NYG,27,21,3,0,3,0,0,1,0
7,2015122005,56717,2015-12-20 12:00:00,15,Sunday,2015,Regular,True,NE,1,...,TEN,16,0,3,7,6,0,0,1,0
8,2014122800,56410,2014-12-28 12:00:00,17,Sunday,2014,Regular,True,ATL,1,...,CAR,34,10,14,10,0,0,0,0,0
9,2015102900,56608,2015-10-29 19:25:00,8,Thursday,2015,Regular,True,NE,1,...,MIA,7,0,0,7,0,0,3,1,0


In [6]:
query = """
select 
*
from games
where
TEAM = '{}'
or
TEAM = '{}'
"""
query = query.format(team_1, team_2)
team_games_df = sqlContext.sql(query)

In [7]:
team_games_df.count()

336

In [8]:
team_games_df.registerTempTable("team_games")

In [9]:
query = """
select
cast(GSIS_ID as char(4))  as GSIS_ID,
ROW_NUMBER()OVER(Partition By TEAM order by GSIS_ID) as row,
TEAM,
SCORE,
OPPONENT_TEAM,
OPPONENT_SCORE
from
team_games
where TEAM = '{team1}'
and 
OPPONENT_TEAM = '{team2}'
UNION ALL
select
cast(GSIS_ID as char(4)) as GSIS_ID,
ROW_NUMBER()OVER(Partition By TEAM order by GSIS_ID) as row,
TEAM,
SCORE,
OPPONENT_TEAM,
OPPONENT_SCORE
from
team_games
where TEAM = '{team2}'
and 
OPPONENT_TEAM = '{team1}'
ORDER BY GSIS_ID
"""
query = query.format(team1=team_1, team2=team_2)
team_opp = sqlContext.sql(query)
team_opp_pd = team_opp.toPandas()


In [10]:
team_opp_pd

,GSIS_ID,row,TEAM,SCORE,OPPONENT_TEAM,OPPONENT_SCORE
0,2009092705,1,ATL,10,NE,26
1,2009092705,1,NE,26,ATL,10
2,2010081951,2,ATL,10,NE,28
3,2010081951,2,NE,28,ATL,10
4,2013092912,3,ATL,23,NE,30
5,2013092912,3,NE,30,ATL,23


In [11]:
import brunel
%brunel data('team_opp_pd') bar x(row) y(SCORE) color(TEAM) tooltip(TEAM, SCORE) sort(row, SCORE) :: width=800, height=500

<IPython.core.display.Javascript object>

In [12]:
query = """
select
GSIS_ID,
SEASON_YEAR,
WEEK,
WON,
SCORE,
OPPONENT_SCORE,
OPPONENT_TEAM,
TEAM,
ROW_NUMBER()OVER(Partition By TEAM order by GSIS_ID) as row
from
games
where TEAM = '{team2}'OR TEAM = '{team1}'
order by
season_year, week
"""
query = query.format(team1=team_1, team2=team_2)
years_df = sqlContext.sql(query)
years_df_pd = years_df.toPandas()



In [13]:
%brunel bar data('years_df_pd') x(row) y(score) interaction(filter)  + bar y(opponent_score) x(row) label(#count) interaction(filter) | bar  split(team) label(team) sum(score) stack color (#selection) interaction(select) :: width=1000, height=500

<IPython.core.display.Javascript object>

In [30]:
query = """
select
team,
a.gsis_id,
season_year,
week,
sum(rushing_yds) as rushing_yds,
sum(passing_yds) as passing_yds
from agg_play a
inner join games g on
    a.gsis_id = g.gsis_id
where g.TEAM = '{team2}' OR g.TEAM = '{team1}'
group by
team,
a.gsis_id,
season_year,
week
order by
season_year,
week
"""
query = query.format(team1=team_1, team2=team_2)
stats_df = sqlContext.sql(query)

stats_df.take(1)

[Row(team=u'NE', gsis_id=u'2009091400', season_year=2009, week=1, rushing_yds=163, passing_yds=590)]

In [32]:
stats_df_pd = stats_df.toPandas()

%brunel data('stats_df_pd') line x(week) y(passing_yds) color(TEAM) tooltip(passing_yds) sort(season_year, week):: width=800, height=500

<IPython.core.display.Javascript object>